In [9]:
import mariadb
from os import getenv

In [10]:
import pandas as pd

In [227]:
class MariaDbSupport:
    
    def __init__(self,user,pwd,host,database,port=3306):
        
        self.user=user
        self.pwd=pwd
        self.host=host
        self.database=database
        self.port=port
        self.dms=False
        self.connected=False
    
    def count(self,table):
        self.cur.execute("SELECT count(*) from " + table)
        o=self.cur.next()
        return o[0]

        
    def debug_mode(self,value:bool=True):
        '''
        Az osztályt debug üzemmódba teszi.(default=True) Több kiírás jelenik neg a kimeneten 
            ha bemeneti értéke False: a debug móde kikapcsolásra kerül
            az osztály létrehozásakor a debug mód ki van kapcsolva 
        '''
        self.dms=value #Debug_mode_state

    def connect(self,db):
        if not self.connected:
            self.conn=mariadb.connect(user=self.user,password=self.pwd,host=self.host,port=self.port,database=db)
            self.cur=self.conn.cursor()
            self.connected=True
        else:
            print("Conencted!!")
    
    def disconnect(self):
        if self.connected:
            self.cur.close()
            self.conn.close()
            self.connected=False
        else:
            print("disconencted")
        
    def to_csv(self,table:str,fname:str):
        import pandas as pd
        
        '''
        MARIADB adatbázis táblából adatok mentése csvbe
        '''
        
        # print("Mongo_start")
        
        if self.dms:
            print("to_csv")
        
        self.cur.execute("SELECT * from "+table)
        description=self.cur.description
        cols=[i[0] for i in  description]


        df  = pd.DataFrame(self.cur)
        df.columns=cols
        if self.dms:
            print(df.head())
        df.to_csv(fname,index=False)
        df=None
        if self.dms:
            print("to_csv exit")
        return        

    
    def upload_from_csv(self,table:str,fname:str,init=False):
        
        '''
        MARIADB  adatbázis táblába collection feltöltése fname csv-ből
        '''
        import mariadb
        import pandas as pd
        
        
        
        if self.dms:
            print("Upload_start")
        
        if not self.connected:
            print("Not Connected!")
            return
        
        if init:
            self.cur.execute("delete from " + table )
            self.conn.commit()
            
            if self.dms:
                print("Init")


        df=pd.read_csv(fname)
        if self.dms:
            print(df.head())

        
        self.cur.execute("SELECT * from "+table)

        description=self.cur.description
        cols=[i[0] for i in  description]
        cols_str=str(cols).replace("'","")
        cols_str=str(cols_str).replace("[","(").replace("]",")")
        cols_orig=cols_str
        cols_str=cols_str.replace("index1","index1")

        if self.dms:
            print(cols_str)

        list_of_dict=df.to_dict('records')

        for i,element in enumerate(list_of_dict):
            
            o=str(list(element.values())).replace("[","(").replace("]",")")
            sql_cmd=f"INSERT INTO {table} {cols_str} VALUES "+o
            if self.dms:
                print(sql_cmd)
            self.cur.execute(sql_cmd)
            if i % 10000==0:
                self.conn.commit()
                if self.dms:
                    print(i, end="_ _")
        self.conn.commit()    
            
        

        

In [194]:
aa=[{"a":12,"b":23},{"a":12,"b":23}]
for i in aa:
    bb=str(list(i.values())).replace("[","(").replace("]",")")

In [186]:
bb

'(12, 23)'

In [228]:
maria_usr=getenv('maria_nlp_user')
maria_pwd=getenv('maria_nlp_pwd')
maria_host="217.144.54.147"
database="NLP_ABB"
table="ABB_file_location"
csvData="E:/Backup/20220506/pdf_file_location.csv"
toARCH="E:/Backup/20220506/MARIADB_pdf_file_location.csv"


mdb=MariaDbSupport(maria_usr,maria_pwd,maria_host,database)
#mdb.connect(database)
#mdb.debug_mode()
#mdb.upload_from_csv(table,csvData,init=True)
#mdb.disconnect()
#mdb.connect(database)
#mdb.debug_mode()
#mdb.to_csv(table,toARCH)
#mdb.disconnect()
#table="ABB_pdf"
#toARCH="E:/Backup/20220506/MARIADB_pdf_metadata.csv"
#mdb.connect(database)
#mdb.debug_mode()
#mdb.to_csv(table,toARCH)
#mdb.disconnect()






In [229]:

table="ABB_pdf"
csvData="E:/Backup/20220506/pdf_metadata.csv"
mdb.connect(database)
mdb.debug_mode(False)
'''
mdb.upload_from_csv(table,csvData,init=True)
'''
print(mdb.count(table))
mdb.disconnect()


1728782


In [16]:
import mariadb
import pandas as pd
fname="E:/corpus/_META_ABB_sentences_20220313_233000.txt"

df=pd.read_csv(fname)

In [202]:
mdb.disconnect()

In [34]:
df.head()

,_id,index,fname,page,pos0,pos1,pos2,pos3
0,0,0,003_IA_Brochure_Leah N. Oxford_OP_20190624_dfw,0,46,109,457,189
1,1,1,003_IA_Brochure_Leah N. Oxford_OP_20190624_dfw,0,376,216,596,373
2,2,2,003_IA_Brochure_Leah N. Oxford_OP_20190624_dfw,0,140,448,362,528
3,3,3,003_IA_Brochure_Leah N. Oxford_OP_20190624_dfw,0,140,544,364,588
4,4,4,003_IA_Brochure_Leah N. Oxford_OP_20190624_dfw,0,140,604,360,696


In [35]:
df.columns=["_id","index1","fname","page","pos0","pos1","pos2","pos3"]

In [204]:

maria_usr=getenv('maria_nlp_user')
maria_pwd=getenv('maria_nlp_pwd')
maria_host="217.144.54.147"
database="NLP_ABB"


In [50]:
list_of_dict=df.to_dict('records')

In [205]:
print(len(list_of_dict))

1728782


In [113]:
list_of_dict[1]

{'_id': 1,
 'index1': 1,
 'fname': '003_IA_Brochure_Leah N. Oxford_OP_20190624_dfw',
 'page': 0,
 'pos0': 376,
 'pos1': 216,
 'pos2': 596,
 'pos3': 373}

In [209]:
conn=mariadb.connect(user=maria_usr,password=maria_pwd,host=maria_host,port=3306,database=database)

In [115]:
'''
cur=conn.cursor()
cur.execute("delete from ABB_pdf ")
conn.commit()
'''

In [210]:
cur=conn.cursor()

In [213]:
cur.execute("SELECT count(*) from ABB_file_location ")

In [215]:
print(ccc)

None


In [219]:
cur.next()

In [153]:
df2=pd.DataFrame(cur)

In [155]:
len(df2)

2038

In [157]:
description=cur.description
cols=[i[0] for i in  description]

In [158]:
cols

['_id', 'md5', 'url', 'fname']

In [144]:

for i in cur:
    print(i)

('621d139751e606667c0905bd', '544B8842628DA3DF7E3714FD68EF50A0', 'D:/ABB/another/2PAA102573-600_A_en_System_800xA_6.0_Device_Management_Device_Library_Wizard.pdf', '2PAA102573-600_A_en_System_800xA_6.0_Device_Management_Device_Library_Wizard')
('621d139751e606667c0905be', '06BBE24147A025DAAB7EED215F849B21', 'D:/ABB/another/Sales Brochure VP800 Rev B.pdf', 'Sales Brochure VP800 Rev B')
('621d139751e606667c0905bf', '4ED4DFEF1E4F871B2E378937A5E3EBC5', 'D:/ABB/another/3BSE080651_en_Freelance_warms_up_Szeged.pdf', '3BSE080651_en_Freelance_warms_up_Szeged')
('621d139751e606667c0905c0', '39ADA4427EA7E922C39DA2E55AAA246A', 'D:/ABB/another/3BUS092064R0001_-_en_Inform_IT_Enterprise_Historian_Brochure.pdf', '3BUS092064R0001_-_en_Inform_IT_Enterprise_Historian_Brochure')
('621d139751e606667c0905c1', 'F1C117AF25BCA6E48923CA825F1C64C3', 'D:/ABB/another/2PAA115334_en_ABB_800xA_Networks_NE870.pdf', '2PAA115334_en_ABB_800xA_Networks_NE870')
('621d139751e606667c0905c2', 'D8F5286F0096FF2C76DFFA6C869603BB

In [111]:
cur.close()

In [112]:
conn.close()

In [118]:
description=cur.description
cols=[i[0] for i in  description]
cols_str=str(cols).replace("'","")
cols_str=str(cols_str).replace("[","(").replace("]",")")


In [119]:
cols_str

'(_id, index1, fname, page, pos0, pos1, pos2, pos3)'

In [120]:
cols

['_id', 'index1', 'fname', 'page', 'pos0', 'pos1', 'pos2', 'pos3']

In [121]:
i=1
o=str([list_of_dict[i][k] for k in cols]).replace("[","(").replace("]",")")

In [122]:
o

"(1, 1, '003_IA_Brochure_Leah N. Oxford_OP_20190624_dfw', 0, 376, 216, 596, 373)"

In [123]:
for i,_ in enumerate(list_of_dict):
    table="ABB_pdf"
    o=str([list_of_dict[i][k] for k in cols]).replace("[","(").replace("]",")")
    sql_cmd=f"INSERT INTO {table} {cols_str} VALUES "+o
    
    cur.execute(sql_cmd)
    if i % 10000==0:
        conn.commit()
        print(i, end="_ _")

0_ _10000_ _20000_ _30000_ _40000_ _50000_ _60000_ _70000_ _80000_ _90000_ _100000_ _110000_ _120000_ _130000_ _140000_ _150000_ _160000_ _170000_ _180000_ _190000_ _200000_ _210000_ _220000_ _230000_ _240000_ _250000_ _260000_ _270000_ _280000_ _290000_ _300000_ _310000_ _320000_ _330000_ _340000_ _350000_ _360000_ _370000_ _380000_ _390000_ _400000_ _410000_ _420000_ _430000_ _440000_ _450000_ _460000_ _470000_ _480000_ _490000_ _500000_ _510000_ _520000_ _530000_ _540000_ _550000_ _560000_ _570000_ _580000_ _590000_ _600000_ _610000_ _620000_ _630000_ _640000_ _650000_ _660000_ _670000_ _680000_ _690000_ _700000_ _710000_ _720000_ _730000_ _740000_ _750000_ _760000_ _770000_ _780000_ _790000_ _800000_ _810000_ _820000_ _830000_ _840000_ _850000_ _860000_ _870000_ _880000_ _890000_ _900000_ _910000_ _920000_ _930000_ _940000_ _950000_ _960000_ _970000_ _980000_ _990000_ _1000000_ _1010000_ _1020000_ _1030000_ _1040000_ _1050000_ _1060000_ _1070000_ _1080000_ _1090000_ _1100000_ _1110

In [145]:
sql_cmd

"INSERT INTO ABB_pdf (_id, index1, fname, page, pos0, pos1, pos2, pos3) VALUES ('621d11a3d90b5e626f8f7149', 1, '003_IA_Brochure_Leah N. Oxford_OP_20190624_dfw', 0, 46.20380020141602, 109.82373809814452, 457.7418212890625, 189.82373046875)"

In [24]:
conn.commit()

In [25]:
conn.close()